In [2]:
!pip install --upgrade pip setuptools wheel


  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)


In [6]:
!pip install spacy --prefer-binary





[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import spacy

ModuleNotFoundError: No module named 'spacy'

In [4]:
import chromadb
import fitz
from sentence_transformers import SentenceTransformer
from pprint import pprint
from pathlib import Path
import os

In [5]:
#Listando arquivos da pasta local "pdf_documents"

pdf_folder_path = "../pdf_documents"
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

print("Arquivos encontrados:")
for pdf in pdf_files:
    print(f"- {pdf}")

Arquivos encontrados:
- EIA_example1.pdf
- RIMA_example1.pdf


In [15]:
#função para extrair o texto do arquivo pdf

import fitz
import re

def extract_clean_text_from_pdf(pdf_path):
    try:
        with fitz.open(pdf_path) as doc:
            raw_text = "\n".join([page.get_text("text") for page in doc])

        lines = raw_text.splitlines()

        ignore_keywords = [
            # Palavras e expressões irrelevantes
            "sumário", "índice", "resumo", "anexo", "figura", "tabela", "referência", "bibliografia",
            "conteúdo", "protocolo", "cetesb", "cpf", "gov", "governo", "secretaria", "manifestação",
            "certidão", "anuência", "art", "processo", "documento", "materializada por", "cbh",
            "condphaat", "comitê", "ficha de caracterização", "decreto de utilidade pública",
            "assinado de forma digital", "declaro que as informações", "assinatura",
            "interessado/reclamado", "investigado", "réu", "instruções", "autenticação mecânica",
            "ficha de compensação", "comprovante de pagamento", "pagamento realizado", "pagador final",
            "identificação no meu comprovante", "tamanho a2", "mapas", "inseridos nas páginas",
            "decisão de diretoria", "conforme o preenchimento dos campos", "limite, conforme o preenchimento",

            # Instruções e títulos técnicos irrelevantes
            "anexar -", "eas -", "rap -", "eia -", 
            "estudo ambiental simplificado", "relatório ambiental preliminar", "estudo de impacto ambiental",
            "caracterização do empreendimento", "diagnóstico - meio físico", "diagnóstico - meio biótico",
            "diagnóstico - meio socioeconômico", "prognóstico e conclusões",
        ]

        filtered_lines = []
        for line in lines:
            cleaned = line.strip()

            if len(cleaned) < 30:
                continue

            # Ignora se linha contém qualquer keyword irrelevante
            if any(keyword in cleaned.lower() for keyword in ignore_keywords):
                continue

            # Ignora PDFs citados
            if cleaned.lower().endswith(".pdf"):
                continue

            # Índice com pontos e número no final
            if re.search(r"\.{3,}\s*\d{1,3}$", cleaned):
                continue

            # Seções numeradas (ex: 7.2.1.8 Texto)
            if re.match(r"^\d+(\.\d+)+\s+", cleaned):
                continue

            # Títulos como "Mapa - ..." ou "GEO - ..."
            if re.match(r"^(Mapa|GEO)\s*-\s*", cleaned):
                continue

            # Títulos administrativos em maiúsculas
            if cleaned.isupper() and len(cleaned.split()) > 2:
                continue

            # Linhas com códigos ou números longos
            if re.search(r"\d{8,}", cleaned):
                continue

            # Linhas com apenas underscores
            if re.fullmatch(r"_+", cleaned):
                continue

            # Datas no padrão dd/mm/aaaa
            if re.search(r"\d{2}/\d{2}/\d{2,4}", cleaned):
                continue

            # Linhas com hash, assinaturas digitais
            if re.search(r"[A-Fa-f0-9]{10,}", cleaned):
                continue

            filtered_lines.append(cleaned)

        final_text = "\n".join(filtered_lines)
        return final_text if final_text.strip() else None

    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {e}")
        return None








In [16]:
# Exemplo de funcionamento da função com o primeiro arquivo da pasta
if pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_files[0]) 
    pdf_text = extract_clean_text_from_pdf(pdf_path)
    print ("Exemplo de texto extraído do primeiro arquivo da pasta")
    print (f"\n Nome do arquivo : {pdf_files[0]}\n")
    print (pdf_text)
else:
    print("Lista de pdf_files vazia")

Exemplo de texto extraído do primeiro arquivo da pasta

 Nome do arquivo : EIA_example1.pdf

Unidade de Recuperação de Energia
Antes da inclusão dos arquivos recomenda-se assistir ao vídeo explicativo, ler a sessão de “Perguntas Frequentes” e verificar a
Este Formulário contém 12 páginas.
Preencher somente os campos pertinentes à solicitação desejada.
Página 1: Orientações gerais e preenchimento dos dados cadastrais da solicitação.
Complementares - RIC e Audiência Pública.
Página 3 até Página 7: Inserção do estudo ambiental a que se refere sua solicitação, existindo como opções o Termo de
Página 11: Informações Georreferenciadas
Link para escolha da Tipologia
Manual de Elaboração de Estudos com AIA
Solicitação de dispensa do pagamento
Solicitação de Parecer Técnico
Requisição de Informações Complementares - RIC
Audiência Pública - Transcrição
Acompanhamento de Licença de Instalação – LI. Deverão ser preenchidos somente os campos pertinentes à solicitação
Relatório de Solicitação de LI


In [14]:
#Extraindo o texto de todos os arquivos da pasta

pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_file)

    print(f"Processando: {pdf_file}...")  # Mostra qual arquivo está sendo lido

    try:
        pdf_texts[pdf_file] = extract_text_from_pdf(pdf_path)
        print(f"Extração concluída: {pdf_file}\n")
    except Exception as e:
        print(f"Erro ao processar {pdf_file}: {e}\n")

print(f"\nOs textos de {len(pdf_texts)} documentos foram extraídos com sucesso.")


Processando: EIA_example1.pdf...
Extração concluída: EIA_example1.pdf

Processando: RIMA_example1.pdf...
Extração concluída: RIMA_example1.pdf


Os textos de 2 documentos foram extraídos com sucesso.


In [15]:
#O erro "MuPDF error: library error: zlib error: invalid distance too far back" geralmente indica que há uma página em branco no documento,
#mas não impede a extração do texto, permitindo a continuação do processamento


In [16]:
# Carregar a chave da API do ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

#Função para dividir os textos em chunks
def split_text_into_chunks(text, chunk_size=512, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    chunks = splitter.split_text(text)
    return chunks

In [18]:
#Função para gerar os embeddings de cada chunk de acordo com a formtação da OPENAI
def generate_embeddings(chunks):
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    return embeddings_model.embed_documents(chunks)

In [19]:
#Gerando os chunks e embeedings para os textos extraídos dos documentos
text_chunks = {}
embeddings = {}

for pdf_file, full_text in pdf_texts.items():
    if not isinstance(full_text, str):  
        print(f"Aviso: O texto extraído de {pdf_file} não é uma string. Convertendo para string vazia.")
        full_text = ""
    
    text_chunks[pdf_file] = split_text_into_chunks(full_text)
    embeddings[pdf_file] = generate_embeddings(text_chunks[pdf_file])
    print(f"Texto do arquivo {pdf_file} dividido em {len(text_chunks[pdf_file])} blocos e embeddings gerados.")

print(f"\nOs textos de {len(pdf_texts)} documentos foram particionados e processados em embeddings.")


Texto do arquivo EIA_example1.pdf dividido em 849 blocos e embeddings gerados.
Texto do arquivo RIMA_example1.pdf dividido em 101 blocos e embeddings gerados.

Os textos de 2 documentos foram particionados e processados em embeddings.


In [20]:
print(type(embeddings))



<class 'dict'>


In [21]:
print(f"Nome dos arquivos processados para retirar os textos: {pdf_texts.keys()}")

Nome dos arquivos processados para retirar os textos: dict_keys(['EIA_example1.pdf', 'RIMA_example1.pdf'])


In [22]:
# Configurando o ChromaDB com persistência
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [23]:
chroma_client.delete_collection(name="pdf_documents")


In [24]:
# Criando a coleção 
collection = chroma_client.get_or_create_collection(
    name="pdf_documents",
    metadata={"hnsw:space": "cosine"},  # Algoritmo de similaridade
    embedding_function=None  # Permite adicionar embeddings manualmente
)


In [25]:

# Adicionando os embeddings ao ChromaDB
doc_id = 1  # Contador único para IDs
total_chunks = 0

for pdf_file, pdf_embeddings in embeddings.items():
    print(f"Salvando documento: {pdf_file} ({len(pdf_embeddings)} chunks)")

    for idx, embedding in enumerate(pdf_embeddings):
        chunk_id = f"{pdf_file}_chunk_{idx}"  # ID único para cada chunk
        collection.add(
            ids=[chunk_id],
            embeddings=[embedding],
            metadatas=[{
                "chunk_index": idx,
                "numeric_id": doc_id,
                "source_file": pdf_file
            }]
        )
        doc_id += 1
        total_chunks += 1

    print(f" -> Documento '{pdf_file}' salvo com sucesso.\n")

print(f"\n{total_chunks} chunks armazenados no ChromaDB.")
print("Todos os documentos foram salvos no banco ChromaDB.")


Salvando documento: EIA_example1.pdf (849 chunks)
 -> Documento 'EIA_example1.pdf' salvo com sucesso.

Salvando documento: RIMA_example1.pdf (101 chunks)
 -> Documento 'RIMA_example1.pdf' salvo com sucesso.


950 chunks armazenados no ChromaDB.
Todos os documentos foram salvos no banco ChromaDB.


In [26]:
print("ids dos documentos:\n")
print(collection.get()["ids"])

ids dos documentos:

['EIA_example1.pdf_chunk_0', 'EIA_example1.pdf_chunk_1', 'EIA_example1.pdf_chunk_2', 'EIA_example1.pdf_chunk_3', 'EIA_example1.pdf_chunk_4', 'EIA_example1.pdf_chunk_5', 'EIA_example1.pdf_chunk_6', 'EIA_example1.pdf_chunk_7', 'EIA_example1.pdf_chunk_8', 'EIA_example1.pdf_chunk_9', 'EIA_example1.pdf_chunk_10', 'EIA_example1.pdf_chunk_11', 'EIA_example1.pdf_chunk_12', 'EIA_example1.pdf_chunk_13', 'EIA_example1.pdf_chunk_14', 'EIA_example1.pdf_chunk_15', 'EIA_example1.pdf_chunk_16', 'EIA_example1.pdf_chunk_17', 'EIA_example1.pdf_chunk_18', 'EIA_example1.pdf_chunk_19', 'EIA_example1.pdf_chunk_20', 'EIA_example1.pdf_chunk_21', 'EIA_example1.pdf_chunk_22', 'EIA_example1.pdf_chunk_23', 'EIA_example1.pdf_chunk_24', 'EIA_example1.pdf_chunk_25', 'EIA_example1.pdf_chunk_26', 'EIA_example1.pdf_chunk_27', 'EIA_example1.pdf_chunk_28', 'EIA_example1.pdf_chunk_29', 'EIA_example1.pdf_chunk_30', 'EIA_example1.pdf_chunk_31', 'EIA_example1.pdf_chunk_32', 'EIA_example1.pdf_chunk_33', 'E

In [27]:
#exemplo de consulta de documento por id numérico nos metadados

result = collection.get(where={"numeric_id": 5})  # Busca pelo ID numérico armazenado como metadado

print(result)


{'ids': ['EIA_example1.pdf_chunk_4'], 'embeddings': None, 'documents': [None], 'uris': None, 'data': None, 'metadatas': [{'chunk_index': 4, 'numeric_id': 5, 'source_file': 'EIA_example1.pdf'}], 'included': [<IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [28]:
from langchain_openai import OpenAIEmbeddings

# 🔹 Criando novamente o gerador de embeddings
openai_embeddings = OpenAIEmbeddings()  # Garante que usa o mesmo modelo dos chunks

# 🔹 Definindo a query de busca
query = "defina o que é uma Área Diretamente Afetada"
query_embedding = openai_embeddings.embed_query(query)  # Gera embedding para a query

# 🔹 Definindo o número de resultados desejados
n = 10

# 🔹 Buscando os N documentos mais relevantes no ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],  
    n_results=n  
)

# 🔹 Exibindo os resultados
print("\n**Resultados da Busca:**")
for i, doc in enumerate(results["metadatas"][0]):
    print(f"{i+1}. Chunk {doc['chunk_index']} (Score: {results['distances'][0][i]})")



**Resultados da Busca:**
1. Chunk 838 (Score: 0.12370055913925171)
2. Chunk 48 (Score: 0.1254950761795044)
3. Chunk 562 (Score: 0.1290644407272339)
4. Chunk 556 (Score: 0.13141751289367676)
5. Chunk 564 (Score: 0.13948553800582886)
6. Chunk 174 (Score: 0.13951945304870605)
7. Chunk 555 (Score: 0.15099716186523438)
8. Chunk 50 (Score: 0.15445005893707275)
9. Chunk 72 (Score: 0.15901178121566772)
10. Chunk 563 (Score: 0.15958094596862793)


In [29]:
# Exibir os primeiros metadados para conferir os índices armazenados
results = collection.get()
print(results["metadatas"][:10])  # Mostra os 10 primeiros registros


[{'chunk_index': 0, 'numeric_id': 1, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 1, 'numeric_id': 2, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 2, 'numeric_id': 3, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 3, 'numeric_id': 4, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 4, 'numeric_id': 5, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 5, 'numeric_id': 6, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 6, 'numeric_id': 7, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 7, 'numeric_id': 8, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 8, 'numeric_id': 9, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 9, 'numeric_id': 10, 'source_file': 'EIA_example1.pdf'}]


In [30]:
def get_chunk_text_by_index(chunk_index):
    results = collection.get(where={"chunk_index": chunk_index})

    if results and "metadatas" in results and results["metadatas"]:
        chunk_id = results["ids"][0]  # Obtém o ID do chunk

        # Buscar o texto diretamente nos chunks armazenados
        for chunks in text_chunks.values():  
            if chunk_index < len(chunks):  
                return chunks[chunk_index]  # Retorna o texto do chunk

    return "Nenhum chunk encontrado com esse índice."

# Teste a função
chunk_index = 564  # Escolha o índice do chunk que deseja visualizar
chunk_text = get_chunk_text_by_index(chunk_index)

print(f"Texto do chunk {chunk_index}:\n{chunk_text}")


Texto do chunk 564:
Para este estudo adotou-se como Área de Influência Direta (AID) um raio de 1.000 
metros no entorno da Área Diretamente Afetada (ADA) pelo empreendimento (distância 
mínima recomendada pela ABNT 13896 para núcleos populacionais), onde se localizam 
basicamente pastagem, área industrial, zona urbana residencial, solo exposto e 
vegetação nativa, conforme figura baixo. 
 Área de Influência Direta (AID): é aquela que absorverá diretamente os


In [ ]:
import openai
import os

openai_embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

def search_query(query: str, n=5):
    query_embedding = openai_embeddings.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    documents = results["documents"][0]

    documentos_filtrados = [doc for doc in documents if doc is not None]

    return "\n\n".join(documentos_filtrados)

#Função do chatbot com RAG
def chatbot(message: str):
    contexto = search_query(message)

    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente o contexto a seguir para responder de forma precisa e técnica.

Contexto extraído de documentos:
{contexto}

Pergunta do usuário:
{message}
"""
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )
    return response.choices[0].message.content

# Loop de interação
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break
    print("Bot:", chatbot(user_input))

In [ ]:
# parar de usar banco de dados em memória (talvez mudar para o postGre ou mongoDb) ou descobrir um jeito de salvar o chorma direito
# utilizar langgraph
#pesquisar sobre multiagentes (react) 
#Procurar forma de limpar documentos
#estudar leitura de imagens (leitura de mapas)
